LSA

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

df = pd.read_csv('data/clean_genai-education_2023-2025.csv') 
print(df.describe())

documents = df['text_clean'].tolist()
texts = [doc.split() for doc in documents]  # tokenized for coherence

       source_name                 date_clean  \
count          206                        206   
unique          33                         65   
top         Zenodo  2025-02-01 00:00:00+00:00   
freq            58                         57   

                                               text_clean        link  
count                                                 206         205  
unique                                                205          89  
top     Advancements in generative #AI have had a sign...  Zenodo.org  
freq                                                    2          58  


In [ ]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(documents)
terms = vectorizer.get_feature_names_out()

NameError: name 'X' is not defined

In [ ]:
def compute_lsa_metrics(X, terms, n_topics, num_words=10):
    lsa = TruncatedSVD(n_components=n_topics, random_state=42)
    lsa.fit(X)
    explained_var = lsa.explained_variance_ratio_.sum()
    return explained_var, lsa

# Plot explained variance for different numbers of topics
topic_range = range(3, 11)
explained_variances = []

for n in topic_range:
    explained_var, lsa_model = compute_lsa_metrics(X, terms, n)
    explained_variances.append(explained_var)

# Plot only explained variance (without coherence)
plt.figure(figsize=(10, 6))
plt.plot(topic_range, explained_variances, marker='s', color='blue', label="Explained Variance")
plt.xlabel("Number of Topics")
plt.ylabel("Explained Variance Ratio")
plt.title("LSA: Explained Variance by Number of Topics")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
summary_data = []
for i in range(lsa_final.n_components):
    top_words = [terms[idx] for idx in lsa_final.components_[i].argsort()[-num_words:]][::-1]
    summary_data.append({
        'Topic': f'Topic {i+1}',
        'Explained Variance': f'{lsa_final.explained_variance_ratio_[i]:.1%}',
        'Top 10 Words': ', '.join(top_words),
        'Avg Document Score': f'{X_lsa[:, i].mean():.3f}',
        'Score Std Dev': f'{X_lsa[:, i].std():.3f}'
    })

summary_df = pd.DataFrame(summary_data)
print("\nLSA TOPIC SUMMARY:")
print("="*170)
print(summary_df.to_string(index=False))

print("\n" + "="*60)
print(f"Generated {n_topics} topics explaining {lsa_final.explained_variance_ratio_.sum():.1%} of total variance")
print("="*60)